# Project 1: Classification Analysis on Textual Data

### Classification Analysis on Textual Data 
#### Spring 2021

### Word Embedding

In [9]:
import nltk
import numpy as np
import pandas as pd
#from scipy import spatial
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.manifold import TSNE
#from sklearn.preprocessing import StandardScaler
#from nltk.corpus import stopwords
#import re
#import string

In [7]:
#Create a key-value dictionary where key is the word and value is the word embedding
embeddings_dict = {}
dimension_of_glove = 300
with open("glove.6B.300d.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

##### QUESTION 8: Read the paper about GLoVE embeddings - found here and answer the following subquestions: answered in report


In [ ]:
print(embeddings_dict['queen']-embeddings_dict['king']-embeddings_dict['wife']+embeddings_dict['husband'])

In [ ]:
print(embeddings_dict['queen']-embeddings_dict['king'])

In [ ]:
print(embeddings_dict['wife']-embeddings_dict['husband'])

In [8]:
print(embeddings_dict['computer'])

[-2.7628e-01  1.3999e-01  9.8519e-02 -6.4019e-01  3.1988e-02  1.0066e-01
 -1.8673e-01 -3.7129e-01  5.9740e-01 -2.0405e+00  2.2368e-01 -2.6314e-02
  7.2408e-01 -4.3829e-01  4.8886e-01 -3.5486e-03 -1.0006e-01 -3.0587e-01
 -1.5621e-01 -6.8136e-02  2.1104e-01  2.9287e-01 -8.8861e-02 -2.0462e-01
 -5.7602e-01  3.4526e-01  4.1390e-01  1.7917e-01  2.5143e-01 -2.2678e-01
 -1.0103e-01  1.4576e-01  2.0127e-01  3.1810e-01 -7.8907e-01 -2.2194e-01
 -2.4833e-01 -1.5103e-02 -2.0050e-01 -2.6441e-02  1.8551e-01  3.3782e-01
 -3.3543e-01  8.6117e-01 -4.7083e-02 -1.7009e-01  3.0438e-01  9.4119e-02
  3.2435e-01 -8.1171e-01  8.8966e-01 -3.9149e-01  1.6828e-01  1.4316e-01
  3.6339e-03 -6.4557e-02  4.5777e-02 -3.2248e-01  4.8943e-02  1.6817e-01
  6.8344e-02  5.4227e-01  1.2493e-01  6.9742e-01 -3.7194e-02  3.3080e-01
 -4.2194e-01  3.3970e-01  2.7646e-01 -1.6003e-02 -2.1827e-01  4.4535e-01
  3.5379e-01 -2.2089e-02  2.1375e-01  4.3267e-01 -3.2897e-01  9.6165e-02
  3.1265e-01 -3.0528e-01  2.6126e-01 -6.5364e-01 -7

## Feature Engineering: Stemming & Lemmatization

##### QUESTION 9: For the binary classification task distinguishing the “Computer Technology” class and “Recreational Activity” class:
1. Describe a feature engineering process that uses GLoVE word embeddings to represent each document. 
2. Select a classifier model, train and evaluate it with your GLoVE-based feature.





##### QUESTION 10: Plot the relationship between the dimension of the pre-trained GLoVE embedding and the resulting accuracy of the model in the classification task. Describe the observed trend. Is this trend expected? Why or why not?

##### QUESTION 11: Compare and contrast the two visualizations. Are there clusters formed in either or both of the plots?

### Multiclass Classification

##### QUESTION 12: In this part, we aim to learn classifiers on the documents belonging to the classes: comp.sys.ibm.pc.hardware, comp.sys.mac.hardware, misc.forsale, soc.religion.christian

Perform Naïve Bayes classification and multiclass SVM classification (with both One VS One and
One VS the rest methods described above) and report the confusion matrix and calculate the
accuracy, recall, precision and F-1 score of your classifiers.
